## Task Description Part A:
Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

## Task Description Part B:
Now there are 10 knots, one still being the head, and the other nine being the tail, while each tail respectively being the head of the following one.

How many positions does the last tail of the rope visit at least once?

## Solution Part A

In [1]:
import numpy as np
from scipy import sparse

import sys
sys.path.append("..")
import lib

In [2]:
def get_max_move(filename : str) -> int:
    lines = lib.read_file(filename)
    max_dist_per_move = 0
    for line in lines:
        dist = int(line[2:])
        if dist > max_dist_per_move:
            max_dist_per_move = dist
    return max_dist_per_move

class Rope():
    def __init__(self, filename : str = None, n_knots : int = 2, verbose : bool = False):
        self.knots = [np.zeros((2), dtype = int) for i in range(n_knots)]
        self.n_knots = n_knots
        self.input = open(filename).read()
        self.filename = filename
        if not (filename is None):
            self.init_sparse_map() # create sparse boolean matrix
            assert self.sparse_map.count_nonzero() == 1
            print(f"Nonzero indices: {self.sparse_map.nonzero()}") if verbose else None
            print("Successfully initialized sparse map.") if verbose else None
        print("Successfully initialized rope instance.") if verbose else None
        del self.input
        return

    def init_sparse_map(self) -> None:
        LEN_FILE = self.input.count('\n') + 1
        MAX_DIST_PER_MOVE = get_max_move(self.filename)
        self.CENTER = LEN_FILE * MAX_DIST_PER_MOVE
        self.sparse_map = sparse.dok_array((2 * self.CENTER + 1, 2* self.CENTER + 1))
        self.sparse_map[self.CENTER, self.CENTER] = 1
        return

    def get_number_of_visited_positions(self) -> int:
        return self.sparse_map.count_nonzero()

    def update_visited_position(self, verbose : bool = False) -> None:
        # update visited position matrix with current position of last knot
        self.sparse_map[self.CENTER + self.knots[-1][0], self.CENTER + self.knots[-1][1]] = 1
        return

    def follow_tail(self, idx_tail : int = 1, verbose : bool = False) -> None:
        idx_head = idx_tail - 1
        pos_diff = self.knots[idx_head] - self.knots[idx_tail]
        assert not(np.any(pos_diff >= 3) or np.any(pos_diff <= -3)), f"Pos_diff = {pos_diff} / Head = {self.knots[idx_head]} / Tail = {self.knots[idx_tail]}"
        if np.all(pos_diff <= 1) and np.all(pos_diff >= -1):
            return # max. 1 step in each row/column -> nothing to do
        else: # change of position == sign(head - tail difference)
            idx_change = np.sign(pos_diff)
        self.knots[idx_tail] += idx_change
        return

    def move_head(self, direction : str, verbose : bool = False) -> None:
        # directions: L -> [-1,0] / R -> [1,0] / D -> [0,-1] / U -> [0,1]
        if direction == 'L':
            idx_change = np.array([-1,0])
        elif direction == 'R':
            idx_change = np.array([1,0])
        elif direction == 'U':
            idx_change = np.array([0,1])
        elif direction == 'D':
            idx_change = np.array([0,-1])
        else:
            ValueError
        self.knots[0] += idx_change
        return

    def parse_motion(self, line : str, verbose : bool = False) -> None:
        # parse motion, then execute it step for step for head, then follow motion with tail
        direction = line[0]
        distance = int(line[2:])
        print(f"{distance} steps in direction {direction}") if verbose else None
        for i in range(distance):
            self.move_head(direction, verbose)
            for idx in range(1, self.n_knots):
                self.follow_tail(idx, verbose)
            self.update_visited_position(verbose)
        return

In [3]:
def test_get_max_move():
    result = get_max_move("test_input.txt")
    assert result == 5, f"get_max_move() faulty... output = {result}"
    print(f"get_max_move() works... output = {result}")
    return
test_get_max_move()


get_max_move() works... output = 5


In [4]:
def compute_partA(filename : str, verbose : bool = False):
    rope = Rope(filename, verbose = True)
    lines = lib.read_file(filename)
    for line in lines:
        rope.parse_motion(line, verbose)
    return rope.get_number_of_visited_positions()

def solve_partA():
    result = compute_partA("test_input.txt", True)
    assert result == 13, f"Part A faulty on test file... output = {result}"
    print("Part A works for test file, moving on to whole input...")
    result = compute_partA("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partA()

Nonzero indices: (array([40], dtype=int32), array([40], dtype=int32))
Successfully initialized sparse map.
Successfully initialized rope instance.
4 steps in direction R
4 steps in direction U
3 steps in direction L
1 steps in direction D
4 steps in direction R
1 steps in direction D
5 steps in direction L
2 steps in direction R
Part A works for test file, moving on to whole input...
Nonzero indices: (array([38000], dtype=int32), array([38000], dtype=int32))
Successfully initialized sparse map.
Successfully initialized rope instance.
Answer: 6236


## Solution Part B

In [5]:
def compute_partB(filename : str, verbose : bool = False):
    rope = Rope(filename, n_knots = 10, verbose = True)
    lines = lib.read_file(filename)
    for line in lines:
        rope.parse_motion(line, verbose)
    return rope.get_number_of_visited_positions()

def solve_partB():
    result = compute_partB("test_input2.txt", True)
    assert result == 36, f"Part B faulty on test file... output = {result}"
    print("Part B works for test file, moving on to whole input...")
    result = compute_partB("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partB()

Nonzero indices: (array([200], dtype=int32), array([200], dtype=int32))
Successfully initialized sparse map.
Successfully initialized rope instance.
5 steps in direction R
8 steps in direction U
8 steps in direction L
3 steps in direction D
17 steps in direction R
10 steps in direction D
25 steps in direction L
20 steps in direction U
Part B works for test file, moving on to whole input...
Nonzero indices: (array([38000], dtype=int32), array([38000], dtype=int32))
Successfully initialized sparse map.
Successfully initialized rope instance.
Answer: 2449
